# Intro to Julia


Why would you want to use Julia? 

1. To go fast: https://julialang.org/benchmarks/
2. To learn how to write better code
3. To use killer apps: http://www.juliaopt.org/notebooks/JuMP-Rocket.html
4. To run ML on TPUs: https://arxiv.org/pdf/1810.09868.pdf

### How do I learn Julia?

1. Lots of great tutorials https://julialang.org/learning/
2. Getting started is easy https://juliabox.com/

## Functions

In [1]:
function plusone(x)
    return x + 1
end

plusone (generic function with 1 method)

In [2]:
plusone(3)

4

In [3]:
plusone(3/4)

1.75

In [4]:
plusone(3//4)

7//4

In [5]:
typeof(3//4)

Rational{Int64}

In [6]:
plusone([1,2,3])

MethodError: MethodError: no method matching +(::Array{Int64,1}, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:292
  +(!Matched::Missing, ::Number) at missing.jl:93
  ...

In [7]:
plusone.([1,2,3])

3-element Array{Int64,1}:
 2
 3
 4

Julia has anonymous (lambda) functions and higher order functions.

In [8]:
f = x -> 2x
map(f, [1,2,3])

3-element Array{Int64,1}:
 2
 4
 6

You can define multiple methods on a single function, by using type constraints

In [9]:
function plusone(f::Function)
    return x->1+f(x)
end


plusone (generic function with 2 methods)

In [10]:
plusone(x->2x)

#5 (generic function with 1 method)

In [11]:
plusone(x->2x)(3)

7

In [12]:
plusone(x->2x).([1,2,3])

3-element Array{Int64,1}:
 3
 5
 7

## Types

Most dynamic languages have you ignore types as much as possible, which is fine for scripting. But, serious software always requires reasoning about types.

In [13]:
struct IntPair
    x::Int
    y::Int
end


In [14]:
IntPair(1,2)

IntPair(1, 2)

In [15]:
IntPair(2,3) + IntPair(4,5)

MethodError: MethodError: no method matching +(::IntPair, ::IntPair)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [16]:
import Base: +
+(a::IntPair, b::IntPair) = IntPair(a.x + b.x, a.y + b.y)

+ (generic function with 164 methods)

In [17]:
IntPair(2,3) + IntPair(4,5)

IntPair(6, 8)

In [18]:
v = [IntPair(1,2), IntPair(2,4), IntPair(3,6), IntPair(4,8)]

4-element Array{IntPair,1}:
 IntPair(1, 2)
 IntPair(2, 4)
 IntPair(3, 6)
 IntPair(4, 8)

In [19]:
dump(v)

Array{IntPair}((4,))
  1: IntPair
    x: Int64 1
    y: Int64 2
  2: IntPair
    x: Int64 2
    y: Int64 4
  3: IntPair
    x: Int64 3
    y: Int64 6
  4: IntPair
    x: Int64 4
    y: Int64 8


In [20]:
length(v)

4

In [21]:
sizeof(v)

64

In [22]:
ptr = pointer(v)

Ptr{IntPair} @0x000000011298c8c0

In [23]:
unsafe_load(ptr, 3)

IntPair(3, 6)

In [24]:
pair = Pair(1,2)

1 => 2

In [25]:
typeof(pair)

Pair{Int64,Int64}

In [26]:
typeof(Pair(1,4//5))

Pair{Int64,Rational{Int64}}

In [27]:
@edit Pair(1,2)

Unknown editor: no line number information passed.
The method is defined at line 15.


## Definition of Pair

```julia
struct Pair{A, B}
    first::A
    second::B
    function Pair{A, B}(@nospecialize(a), @nospecialize(b)) where {A, B}
        @_inline_meta
        # if we didn't inline this, it's probably because the callsite was actually dynamic
        # to avoid potentially compiling many copies of this, we mark the arguments with `@nospecialize`
        # but also mark the whole function with `@inline` to ensure we will inline it whenever possible
        # (even if `convert(::Type{A}, a::A)` for some reason was expensive)
        return new(a, b)
    end
end
Pair(a, b) = Pair{typeof(a), typeof(b)}(a, b)
const => = Pair
```


"""
    Pair(x, y)
    x => y

Construct a `Pair` object with type `Pair{typeof(x), typeof(y)}`. The elements
are stored in the fields `first` and `second`. They can also be accessed via
iteration.

See also: [`Dict`](@ref)

## Examples


```jldoctest
julia> p = "foo" => 7
"foo" => 7

julia> typeof(p)
Pair{String,Int64}

julia> p.first
"foo"

julia> for x in p
           println(x)
       end
foo
7
```

## More functions
```julia

eltype(p::Type{Pair{A, B}}) where {A, B} = Union{A, B}
iterate(p::Pair, i=1) = i > 2 ? nothing : (getfield(p, i), i + 1)
indexed_iterate(p::Pair, i::Int, state=1) = (getfield(p, i), i + 1)

hash(p::Pair, h::UInt) = hash(p.second, hash(p.first, h))

==(p::Pair, q::Pair) = (p.first==q.first) & (p.second==q.second)
isequal(p::Pair, q::Pair) = isequal(p.first,q.first) & isequal(p.second,q.second)

isless(p::Pair, q::Pair) = ifelse(!isequal(p.first,q.first), isless(p.first,q.first),
                                                             isless(p.second,q.second))
getindex(p::Pair,i::Int) = getfield(p,i)
getindex(p::Pair,i::Real) = getfield(p, convert(Int, i))
reverse(p::Pair{A,B}) where {A,B} = Pair{B,A}(p.second, p.first)

firstindex(p::Pair) = 1
lastindex(p::Pair) = 2
length(p::Pair) = 2
first(p::Pair) = p.first
last(p::Pair) = p.second

convert(::Type{Pair{A,B}}, x::Pair{A,B}) where {A,B} = x
function convert(::Type{Pair{A,B}}, x::Pair) where {A,B}
    Pair{A,B}(convert(A, x[1]), convert(B, x[2]))
end

promote_rule(::Type{Pair{A1,B1}}, ::Type{Pair{A2,B2}}) where {A1,B1,A2,B2} =
    Pair{promote_type(A1, A2), promote_type(B1, B2)}
```

## Abstract Types

Types come in two forms, abstract types which can be extended, but not instantiated and concrete types which can be instantiated but not extended.



In [28]:
abstract type Person end
abstract type Color end
abstract type AbstractTeam end
abstract type SportsTeam <: AbstractTeam end
struct FootballTeam <: SportsTeam
    players::AbstractArray{Person}
    coach::Person
    name::String
    colors::Pair{Color, Color}
end


## Macros
You can define new syntax in julia using macros, which are function that take source code as their argument.

In [29]:
let
    i = 0 
    while i < 10
        print(i," ")
        i = i + 1
    end
end

0 1 2 3 4 5 6 7 8 9 

In [30]:
until i == 10
    print(i, " ")
end

LoadError: syntax: extra token "i" after end of expression

In [31]:
macro until(condition, block)
    quote
        while true
            $(esc(block))
            if $(esc(condition))
                break
            end
        end
    end
end

@until (macro with 1 method)

In [32]:
let 
    i = 0
    @until i >= 10 begin
    print(i, " ")
    i += 1
end
end

0 1 2 3 4 5 6 7 8 9 

### Horner's rule

You can save a lot of computation with macros.
```julia
"""
    @horner(x, p...)
    Evaluate p[1] + x * (p[2] + x * (....)), i.e. a polynomial via Horner's rule
"""
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end
```

In [33]:
using Base.Math
@macroexpand Math.@horner 5 1 2 3

quote
    #4#t = 5
    (Base.Math.muladd)(#4#t, (Base.Math.muladd)(#4#t, 3, 2), 1)
end

In [34]:
using BenchmarkTools
@btime Math.@horner(5, 1,2,3,4,5,6,7,8)

  0.030 ns (0 allocations: 0 bytes)


756836

In [35]:
@btime 1 + 2*5 + 3*5^2 + 4*5^3 + 5*5^4 + 6*5^5 + 7*5^6 + 8*5^7

  9.775 ns (0 allocations: 0 bytes)


756836

In [36]:
p = [1,2,3,4,5,6,7,8]
function poly(p)
    y = 0
    for i in 1:8
        y += p[i] * 5^i
    end
end
poly(p)
@btime poly(p)

  34.592 ns (0 allocations: 0 bytes)


## Understanding Performance

Julia is fast because it is not a dynamic language, it dynamically compiles static code.

In [37]:
@code_llvm 5+6


; Function +
; Location: int.jl:53
define i64 @"julia_+_35442"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


In [38]:
function loopit() 
    j = 0
    for i in 1:10
        j += i
    end
    return j
end

@code_native loopit()

	.section	__TEXT,__text,regular,pure_instructions
; Function loopit {
; Location: In[38]:2
	movl	$55, %eax
	retl
	nopw	%cs:(%eax,%eax)
;}


In [39]:
function loopit(n::Int) 
    j = 0
    for i in 1:n
        j += i
    end
    return j
end

@code_native loopit(15)

	.section	__TEXT,__text,regular,pure_instructions
; Function loopit {
; Location: In[39]:3
; Function Colon; {
; Location: range.jl:5
; Function Type; {
; Location: range.jl:255
; Function unitrange_last; {
; Location: range.jl:260
; Function >=; {
; Location: operators.jl:333
; Function <=; {
; Location: In[39]:2
	decl	%eax
	testl	%edi, %edi
;}}}}}
	jle	L32
	decl	%eax
	leal	-1(%edi), %edx
	decl	%eax
	leal	-2(%edi), %eax
	mulxl	%eax, %eax, %ecx
	decl	%eax
	shldl	$63, %eax, %ecx
	decl	%eax
	leal	(%ecx,%edi,2), %eax
	decl	%eax
	addl	$-1, %eax
; Location: In[39]:6
	retl
L32:
	xorl	%eax, %eax
; Location: In[39]:6
	retl
	nopw	%cs:(%eax,%eax)
;}


<img href="./">

<img src="./godbolt_loopit.png">